# Análise Trabalho T1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
import scipy.optimize as spo
import sympy as sp
np.set_printoptions(suppress=True, linewidth= 500)
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

### Funções

In [3]:
def makeplot(data, dt_label, color_mk, *errors, bars=False,):
    data_x, data_y = data[0], data[1]
    fig , ax = plt.subplots( figsize = figsize, dpi = dpi  ) 
    ax.legend( fontsize = fontsize)
    ax.set_xlabel( xlabel , fontsize = fontsize )
    ax.set_ylabel( ylabel , fontsize = fontsize )
    if ticks:
        ax.set_xticks( xticks )
        ax.set_yticks( yticks )
        ax.set_xlim( xticks[0] , xticks[-1])
        ax.set_ylim( yticks[0] , yticks[-1])
    plt.grid( alpha = alpha_grid )
    plot_data = ax.plot( data_x , data_y , marker_type ,
                    label      = dt_label , 
                    markersize = markersize )
    plot_data[0].set_color(color_mk )
    if bars:
        errorbar = plt.errorbar( data_x , data_y ,
                                yerr       = errors[0][1] ,
                                xerr       = errors[0][0] , 
                                color      = color_bars ,
                                ls         ='' , 
                                alpha      = alpha_bars ,
                                elinewidth = thickness,
                                capsize    = head_size , 
                                capthick   = 0.2 )
    plt.legend( fontsize = fontsize )
    plt.tight_layout()
    return fig, ax

def add_points(data, dt_label, color_mk, figure,*errors, bars= False):
    data_x, data_y = data[0], data[1]
    fig, ax = figure[0], figure[1]
    plot_data = ax.plot( data_x , data_y , marker_type ,
                    label      = dt_label , 
                    markersize = markersize )
    plot_data[0].set_color(color_mk )
    if bars:
        errorbar = plt.errorbar( data_x , data_y ,
                                yerr       = errors[0][1] ,
                                xerr       = errors[0][0] , 
                                color      = color_bars ,
                                ls         ='' , 
                                alpha      = alpha_bars ,
                                elinewidth = thickness,
                                capsize    = head_size , 
                                capthick   = 0.2 )
    ax.legend( fontsize = fontsize )

def add_fit( ax , x , coef , f_fit , label="Fit",linestyle = "k--"):
    y_fit     = f_fit( x ,*coef )
    linewidth = 1
    alpha     = 0.5 
    if label != False:
        ax.plot( x , y_fit , linestyle , label = label, linewidth = linewidth , alpha = alpha  )
        ax.legend(  fontsize = fontsize )
    else:
         ax.plot( x , y_fit , linestyle , linewidth = linewidth , alpha = alpha )
    
    return ax , y_fit

def add_residues( x , y_fit , y , color, in_fig , *args ):
    if not in_fig: 
        fig , ax = plt.subplots( figsize = (figsize))
        ax.set_xlabel( xlabel )
        ax.set_ylabel( "Resíduos" )
        ax.legend( fontsize = 15)
    
    else : 
        fig = args[ 0 ]
        ax = args[ 1 ]
        loc= args[2]
        ax = inset_axes(ax, width="30%", height="30%", loc=loc)
    
    res       = y - y_fit
    
    global max_res
    global min_res
    max_res, min_res = max(max_res, np.max(res)), min(min_res,np.min(res))
    ax.scatter( x , res ,color = color, marker='.' )
    if ticks:
        ax.set_xticks( xticks )
        ax.set_yticks( yticks )
        ax.set_xlim( ( xticks[0] , xticks[-1]) )
        ax.set_ylim( ( yticks[0] , yticks[-1]) )
    ax.grid(True)
    ax.set_title("Resíduos")
    if in_fig:
        return ax     
    return fig , ax  

colors=['blue', 'orange', 'green', 'magenta', 'purple']
max_res, min_res = 0, 0
# para guardar a imagem 
#fig.savefig( fig_name + ".png" )

### Formatação de gráfico 

Fica aqui o código genérico para fazer um gráfico maravilhoso 

##### Código para mudar 

In [2]:
# --- Formatação da figura ---
cm          = 1/2.54  # centimeters in inches
figsize     = ( 12*cm, 10*cm )
dpi         = 100

# --- Texto que apareça no gráfico ---
fontsize        = 8
legend_label    = "Pontos Experimentais"
xlabel          = ""
ylabel          = "$\|\epsilon| $(mV)"

# --- Formação dos pontos ---
marker_type = "."
markersize  = 2.5
color_mk    = "lime"


# --- formatação da escala do gráfico ---
xticks      = np.linspace( - 1 , 1 , 10 )
yticks      = np.linspace( -20 , 20 , 9)

# --- Transparência da grid ---
alpha_grid = 0.5 

# --- Formatação barras de erro --- 
color_bars    = "k"
alpha_bars    = 0.5
thickness     = 1 
head_size     = 1 
head_thick    = 0.2 

# --- Informação para guardar a imagem ---
fig_name = "fig1" #sem o png!


# --- data para dar plot ---
#data_x = ...
#data_y = ...
#u x    = ...
#u_y    = ...

## Import da data

Nota -- os dados estão divididos em colunas principais para as series de medidas.
* Primeira coluna é para os dados do primeiro aquecimento
* a segunda é para o primeiro arrefecimento
* a terceira para o segundo aquecimento 
* a quarta é para o segundo arrefecimento 
* a quinta é para os dados das primeiras medições, onde nos enganamos. Não medimos a corrente do diodo, apenas temos medidas da resistência, mas como são à temperatura ambiente, continuam a ser bons valores. 
* ainda temos duas colunas onde temos apenas dois valores onde temos as incertezas das medidas da resistência e da temperatura ( ambos os valores são 0.1 nas suas dimensões ) . 
* as colunas são compostas por outras colunas, Temperatura, Resistência, Corrente e incerteza da corrente. 

In [2]:
data = np.genfromtxt("T1_CSV.csv", delimiter = ",", skip_header=1)

In [5]:
# dividir a data : 
# primeiro aquecimento
T1 , R1 , I1 , uI1 = data[ : , 0 ] , data[ : , 1 ] , data[ : , 2 ] , data[ : , 3 ]

# primeiro arrefecimento 
T2 , R2 , I2 , uI2 = data[ : , 4 ] , data[ : , 5 ] , data[ : , 6 ] , data[ : , 7 ]

# segundo aquecimento
T3 , R3 , I3 , uI3 = data[ : , 8 ] , data[ : , 9 ] , data[ : , 10 ] , data[ : , 11 ]

# segundo arrefecimento 
T4 , R4 , I4 , uI4 = data[ : , 12 ] , data[ : , 13 ] , data[ : , 14 ] , data[ : , 15 ]

# primeira serie de medidas, ignorar I e uI
T5 , R5 , I5 , uI5 = data[ : , 16 ] , data[ : , 17 ] , data[ : , 18 ] , data[ : , 19 ]

# incertezas que foram constantes, são floats, temos de dar reshape em cada data set
# usar -> uRx = np.ones( len( Tx ) ) * uR por exemplo
uR , uT = data[ 0 , 20 ] , data[ 0 , 21 ]

É só preciso fazer ajustes lineares, por isso usar ``sps.linregress``